In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from datetime import datetime as dt
import itertools


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm


from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

from keras.utils import np_utils

%matplotlib inline

Using TensorFlow backend.


# Load the Dataset

In [2]:
df = pd.read_csv("../datasets/FIFA 1990-2018 - Cleaned.csv")
squads = pd.read_csv("../datasets/New_data/2018 FIFA World Cup Squads.csv")
fifa18 = pd.read_csv("../datasets/New_data/complete.csv")
distance = pd.read_csv("../datasets/New_data/distance.csv", error_bad_lines=False)
fifa_ranking = pd.read_csv("../datasets/New_data/fifa_ranking.csv")

b'Skipping line 31: expected 4 fields, saw 5\nSkipping line 63: expected 4 fields, saw 5\nSkipping line 85: expected 4 fields, saw 5\nSkipping line 102: expected 4 fields, saw 5\nSkipping line 173: expected 4 fields, saw 5\nSkipping line 174: expected 4 fields, saw 5\nSkipping line 196: expected 4 fields, saw 5\nSkipping line 203: expected 4 fields, saw 5\nSkipping line 204: expected 4 fields, saw 5\nSkipping line 207: expected 4 fields, saw 5\nSkipping line 211: expected 4 fields, saw 5\nSkipping line 213: expected 4 fields, saw 5\n'


# Add Ranking for 2018

In [3]:
ranking_2018 = fifa_ranking[(fifa_ranking['rank_date']>"2018-04-01") & (fifa_ranking['rank_date']<"2018-05-01")]
ranking_2018.shape

(211, 17)

In [4]:
ranking_2018

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date,Unnamed: 16
57160,1,Germany,GER,1532.68,1609.0,0,784.89,784.89,804.19,402.09,463.51,139.05,1033.19,206.64,UEFA,2018-04-12,NaN
57161,2,Brazil,BRA,1384.24,1489.0,0,560.52,560.52,990.59,495.29,565.90,169.77,793.26,158.65,CONMEBOL,2018-04-12,NaN
57162,3,Belgium,BEL,1345.65,1337.0,2,684.48,684.48,594.99,297.50,592.33,177.70,929.90,185.98,UEFA,2018-04-12,NaN
57163,4,Portugal,POR,1306.35,1360.0,-1,687.22,687.22,658.47,329.23,576.48,172.94,584.73,116.95,UEFA,2018-04-12,NaN
57164,5,Argentina,ARG,1253.61,1359.0,-1,418.94,418.94,839.74,419.87,769.42,230.83,919.87,183.97,CONMEBOL,2018-04-12,NaN
57165,6,Switzerland,SUI,1179.21,1197.0,2,594.33,594.33,716.99,358.49,390.85,117.26,545.66,109.13,UEFA,2018-04-12,NaN
57166,7,France,FRA,1166.42,1185.0,2,517.68,517.68,828.04,414.02,347.19,104.16,652.83,130.57,UEFA,2018-04-12,NaN
57167,8,Spain,ESP,1161.59,1228.0,-2,571.55,571.55,593.19,296.59,684.65,205.39,440.29,88.06,UEFA,2018-04-12,NaN
57168,9,Chile,CHI,1145.74,1161.0,1,435.51,435.51,714.45,357.22,850.55,255.17,489.25,97.85,CONMEBOL,2018-04-12,NaN
57169,10,Poland,POL,1117.94,1228.0,-4,499.90,499.90,764.51,382.25,450.51,135.15,503.18,100.64,UEFA,2018-04-12,NaN


In [5]:
# Create new columns with ranking position of each team
df.insert(2, 'rank_Team1', df['Team1'].map(ranking_2018.set_index('country_full')['rank']))
df.insert(3, 'rank_Team2', df['Team2'].map(ranking_2018.set_index('country_full')['rank']))

df.insert(4, 'rank_change_Team1', df['Team1'].map(ranking_2018.set_index('country_full')['rank_change']))
df.insert(5, 'rank_change_Team2', df['Team2'].map(ranking_2018.set_index('country_full')['rank_change']))

In [6]:
df.head()

,Team1,Team2,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,66.0,70.0,-3.0,-1.0,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,46.0,17.0,-2.0,5.0,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,42.0,36.0,0.0,-3.0,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,4.0,8.0,-1.0,-2.0,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,7.0,40.0,2.0,-3.0,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


# Add Team features

In [7]:
# #AVG Age of each team in each position
age_avg=squads.groupby(["Team","Position"])['Age'].sum()/squads.groupby(["Team","Position"])['Age'].size()
age_avg = age_avg.to_frame()

age_avg['Team_name'] = age_avg.index.get_level_values(0).tolist()
age_avg['pos'] = age_avg.index.get_level_values(1).tolist()

In [8]:
age_avg = age_avg.replace({"Iran":"IR Iran"})
age_avg = age_avg.replace({"Korea Republic":"South Korea"})
age_avg.head()

Age  Team_name pos
Team      Position                          
Argentina DF        29.111111  Argentina  DF
          FW        28.500000  Argentina  FW
          GK        33.000000  Argentina  GK
          MF        26.428571  Argentina  MF
Australia DF        27.428571  Australia  DF

In [9]:
age_DF = age_avg.loc[age_avg['pos'] == 'DF']
age_FW = age_avg.loc[age_avg['pos'] == 'FW']
age_GK = age_avg.loc[age_avg['pos'] == 'GK']
age_MF = age_avg.loc[age_avg['pos'] == 'MF']

In [10]:
df.head()

,Team1,Team2,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,66.0,70.0,-3.0,-1.0,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,46.0,17.0,-2.0,5.0,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,42.0,36.0,0.0,-3.0,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,4.0,8.0,-1.0,-2.0,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,7.0,40.0,2.0,-3.0,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


In [11]:
df.insert(2, 'Team1_avg_age_DF', df['Team1'].map(age_DF.set_index('Team_name')['Age']))
df.insert(3, 'Team1_avg_age_FW', df['Team1'].map(age_FW.set_index('Team_name')['Age']))
df.insert(4, 'Team1_avg_age_GK', df['Team1'].map(age_GK.set_index('Team_name')['Age']))
df.insert(5, 'Team1_avg_age_MF', df['Team1'].map(age_MF.set_index('Team_name')['Age']))

In [12]:
df.insert(6, 'Team2_avg_age_DF', df['Team2'].map(age_DF.set_index('Team_name')['Age']))
df.insert(7, 'Team2_avg_age_FW', df['Team2'].map(age_FW.set_index('Team_name')['Age']))
df.insert(8, 'Team2_avg_age_GK', df['Team2'].map(age_GK.set_index('Team_name')['Age']))
df.insert(9, 'Team2_avg_age_MF', df['Team2'].map(age_MF.set_index('Team_name')['Age']))

In [13]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


In [14]:
#AVG Caps of each team per position
caps_avg = squads.groupby(["Team","Position"])['Caps'].sum()/squads.groupby(["Team","Position"])['Caps'].size()
caps_avg = caps_avg.to_frame()

caps_avg['Team_name'] = caps_avg.index.get_level_values(0).tolist()
caps_avg['pos'] = caps_avg.index.get_level_values(1).tolist()

In [15]:
caps_avg = caps_avg.replace({"Iran":"IR Iran"})
caps_avg = caps_avg.replace({"Korea Republic":"South Korea"})
caps_avg.head()

Caps  Team_name pos
Team      Position                          
Argentina DF        34.444444  Argentina  DF
          FW        73.000000  Argentina  FW
          GK         3.000000  Argentina  GK
          MF        32.714286  Argentina  MF
Australia DF        22.428571  Australia  DF

In [16]:
caps_DF = caps_avg.loc[age_avg['pos'] == 'DF']
caps_FW = caps_avg.loc[age_avg['pos'] == 'FW']
caps_GK = caps_avg.loc[age_avg['pos'] == 'GK']
caps_MF = caps_avg.loc[age_avg['pos'] == 'MF']

In [17]:
df.insert(10, 'Team1_avg_caps_DF', df['Team1'].map(caps_DF.set_index('Team_name')['Caps']))
df.insert(11, 'Team1_avg_caps_FW', df['Team1'].map(caps_FW.set_index('Team_name')['Caps']))
df.insert(12, 'Team1_avg_caps_GK', df['Team1'].map(caps_GK.set_index('Team_name')['Caps']))
df.insert(13, 'Team1_avg_caps_MF', df['Team1'].map(caps_MF.set_index('Team_name')['Caps']))

df.insert(14, 'Team2_avg_caps_DF', df['Team2'].map(caps_DF.set_index('Team_name')['Caps']))
df.insert(15, 'Team2_avg_caps_FW', df['Team2'].map(caps_FW.set_index('Team_name')['Caps']))
df.insert(16, 'Team2_avg_caps_GK', df['Team2'].map(caps_GK.set_index('Team_name')['Caps']))
df.insert(17, 'Team2_avg_caps_MF', df['Team2'].map(caps_MF.set_index('Team_name')['Caps']))

In [18]:
for c in df.keys():
    print(c)


Team1
Team2
Team1_avg_age_DF
Team1_avg_age_FW
Team1_avg_age_GK
Team1_avg_age_MF
Team2_avg_age_DF
Team2_avg_age_FW
Team2_avg_age_GK
Team2_avg_age_MF
Team1_avg_caps_DF
Team1_avg_caps_FW
Team1_avg_caps_GK
Team1_avg_caps_MF
Team2_avg_caps_DF
Team2_avg_caps_FW
Team2_avg_caps_GK
Team2_avg_caps_MF
rank_Team1
rank_Team2
rank_change_Team1
rank_change_Team2
Rounds
Stadium
City
Time
Day
Month
Year
Score1
Score2
Remarks


# change Rounds

In [19]:
rounds = df.Rounds
rounds.unique()
rounds_dict = {"Group Phase":1, "Round of 16":2, "Quarter Finals":3, "Semi Finals":4, "Play-off for third place":5, "Finals":6}
rounds_encoded = []

for i in range(len(df)):
    rounds_encoded.append(rounds_dict[list(df.Rounds)[i]])
    
df["Rounds"] = rounds_encoded

In [20]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,1,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,1,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,1,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,1,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,1,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


# Player Features

In [21]:
fifa18 = fifa18.replace({"Korea Republic":"South Korea"})
wc_player_stats_18 = fifa18[fifa18["name"].isin(squads.Player) | fifa18["full_name"].isin(squads.Player)]
wc_fifa18_stats = fifa18[fifa18.nationality.isin(squads.Team)]

In [22]:
wc_fifa18_stats.replace([False,"False",True,"True"],[0,0,1,1])
unused_var=[]
for i in fifa18.columns:
    if "prefers" in i:
        unused_var.append(i)
wc_fifa18_stats = wc_fifa18_stats.drop(["club_logo","ID","real_face","birth_date","flag","photo"],axis=1)
wc_fifa18_stats = wc_fifa18_stats.drop(unused_var,axis=1)
grouped_wc_fifa18_stats = wc_fifa18_stats.groupby(['nationality']).apply(lambda x: (x.sort_values('overall',ascending=False)).head(30).mean()).sort_values('potential',ascending=True)
grouped_wc_fifa18_stats.head()

,special,age,height_cm,weight_kg,eur_value,eur_wage,eur_release_clause,overall,potential,pac,...,playmaker_speciality,engine_speciality,distance_shooter_speciality,crosser_speciality,free_kick_specialist_speciality,tackling_speciality,tactician_speciality,acrobat_speciality,strength_speciality,clinical_finisher_speciality
nationality,,,,,,,,,,,,,,,,,,,,,
Panama,1617.375000,25.812500,181.750000,76.500000,8.378125e+05,3312.500000,1.344812e+06,65.812500,69.437500,66.062500,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.062500,0.0
Tunisia,1677.966667,27.266667,182.700000,76.666667,2.261167e+06,11700.000000,2.739000e+06,68.966667,71.533333,66.200000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.033333,0.0
Costa Rica,1695.966667,26.433333,181.000000,75.400000,3.196167e+06,18033.333333,5.612536e+06,69.900000,72.000000,73.400000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.066667,0.0
Iceland,1771.000000,27.233333,184.066667,78.200000,2.607500e+06,16700.000000,4.797357e+06,70.033333,72.166667,67.766667,...,0.0,0.1,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.000000,0.0
Peru,1651.965517,25.517241,178.931034,73.896552,2.122069e+06,6620.689655,5.940867e+06,69.137931,72.689655,69.551724,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.034483,0.000000,0.0


In [23]:
grouped_wc_fifa18_stats['Team_name'] = grouped_wc_fifa18_stats.index.get_level_values(0).tolist()

In [24]:
grouped_wc_fifa18_stats = grouped_wc_fifa18_stats.replace({"Iran":"IR Iran"})
grouped_wc_fifa18_stats.head()

,special,age,height_cm,weight_kg,eur_value,eur_wage,eur_release_clause,overall,potential,pac,...,engine_speciality,distance_shooter_speciality,crosser_speciality,free_kick_specialist_speciality,tackling_speciality,tactician_speciality,acrobat_speciality,strength_speciality,clinical_finisher_speciality,Team_name
nationality,,,,,,,,,,,,,,,,,,,,,
Panama,1617.375000,25.812500,181.750000,76.500000,8.378125e+05,3312.500000,1.344812e+06,65.812500,69.437500,66.062500,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.062500,0.0,Panama
Tunisia,1677.966667,27.266667,182.700000,76.666667,2.261167e+06,11700.000000,2.739000e+06,68.966667,71.533333,66.200000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.033333,0.0,Tunisia
Costa Rica,1695.966667,26.433333,181.000000,75.400000,3.196167e+06,18033.333333,5.612536e+06,69.900000,72.000000,73.400000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.066667,0.0,Costa Rica
Iceland,1771.000000,27.233333,184.066667,78.200000,2.607500e+06,16700.000000,4.797357e+06,70.033333,72.166667,67.766667,...,0.1,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.000000,0.0,Iceland
Peru,1651.965517,25.517241,178.931034,73.896552,2.122069e+06,6620.689655,5.940867e+06,69.137931,72.689655,69.551724,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.034483,0.000000,0.0,Peru


In [25]:
df.head() 

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,1,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,1,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,1,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,1,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,1,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


In [26]:
for key in grouped_wc_fifa18_stats.keys():
    if key != 'Team_name':
        df.insert(12, 'Team2_'+key, df['Team2'].map(grouped_wc_fifa18_stats.set_index('Team_name')[key]))
        df.insert(12, 'Team1_'+key, df['Team1'].map(grouped_wc_fifa18_stats.set_index('Team_name')[key]))
        

In [27]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,1,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,1,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,1,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,1,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,1,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


# code country name

In [28]:
countries = list(set(df["Team1"]).union(set(df["Team2"])))
countries_dict = {}
for i in range(len(countries)):
    countries_dict[countries[i]] = i
    
Team1_encoded = []
Team2_encoded = []

for i in range(len(df)):
    Team1_encoded.append(countries_dict[list(df.Team1)[i]])
    Team2_encoded.append(countries_dict[list(df.Team2)[i]])
    
df["Team1"] = Team1_encoded
df["Team2"] = Team2_encoded

In [29]:
#Adding who is the winner 
winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
    else:
        winner.append(-1)
df['winning_team'] = winner

df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks,winning_team
0,57,24,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN,57
1,63,47,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN,47
2,34,29,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN,29
3,17,41,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...,-1
4,58,15,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN,58


In [30]:

df = df.drop(['Score1','Score2','Remarks',"Stadium","City"],axis=1)
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Time,Day,Month,Year,winning_team
0,57,24,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,66.0,70.0,-3.0,-1.0,1,18:00,14,6,2018,57
1,63,47,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,46.0,17.0,-2.0,5.0,1,17:00,15,6,2018,47
2,34,29,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,42.0,36.0,0.0,-3.0,1,18:00,15,6,2018,29
3,17,41,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,4.0,8.0,-1.0,-2.0,1,21:00,15,6,2018,-1
4,58,15,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,7.0,40.0,2.0,-3.0,1,13:00,16,6,2018,58


In [31]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Time,Day,Month,Year,winning_team
0,57,24,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,66.0,70.0,-3.0,-1.0,1,18:00,14,6,2018,57
1,63,47,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,46.0,17.0,-2.0,5.0,1,17:00,15,6,2018,47
2,34,29,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,42.0,36.0,0.0,-3.0,1,18:00,15,6,2018,29
3,17,41,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,4.0,8.0,-1.0,-2.0,1,21:00,15,6,2018,-1
4,58,15,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,7.0,40.0,2.0,-3.0,1,13:00,16,6,2018,58


# Duplicate

In [ ]:
#we should change all respective column for duplicate
# for index,row in df.iterrows():
#     df.set_value(index+488,["Team1","Team2","Rounds","Day","Month","Year","Score1","Score2","winning_team","avg_team1","avg_team2"],[row["Team2"],row["Team1"],row["Rounds"],row["Day"],row["Month"],row["Year"],row["Score2"],row["Score1"],2 if row["winning_team"]==1 else 1 if row["winning_team"]==2 else 0,row['avg_team2'],row['avg_team1']])
# df.shape

# train & test

In [ ]:
X_train = df.loc[df['Rounds'] == 1]
X_test = df.loc[df['Rounds'] != 1]



In [ ]:
y_train = X_train["winning_team"]
y_test = X_test["winning_team"]

In [ ]:
X_train = X_train.drop(["Rounds","winning_team","Time"],axis=1)
X_test = X_test.drop(["Rounds","winning_team","Time"],axis=1)